### Word level text generation

In this notebook we are going to learn how to perform a word level text generation in keras. In this notebook we are going to create a model that will be able to generate news headlines using [New York Times Comment and Headlines](https://www.kaggle.com/aashita/nyt-comments) dataset wich i got from kaggle.

I've already downloaded the dataset and loaded so I will load it from here eaisly.

### Mounting drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports


In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

import numpy as np
import tensorflow as tf
import pandas as pd

import os, time, math, string

np.__version__

'1.19.5'

#### Loading the data

In [3]:
base_path = '/content/drive/My Drive/NLP Data/text-gen/New York Times Comments'
os.path.exists(base_path)

True

In [4]:
%%time
all_headlines = []
for filename in os.listdir(base_path):
  if 'Articles' in filename:
    df = pd.read_csv(os.path.join(base_path, filename))
    all_headlines.extend(list(
        df.headline.values
    ))
all_headlines = [h for h in all_headlines if h != "Unknown"]
print(len(all_headlines))

8603
CPU times: user 92.4 ms, sys: 17.2 ms, total: 110 ms
Wall time: 185 ms


In [5]:
all_headlines[:15]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.',
 'In Berlin, artists find a home',
 'The Right Stuff',
 'Jimmy Carter Knows What North Korea Wants',
 'The Truth Is Out There',
 'New Jersey Ruling Could Reignite Battle Over Church-State Separation']

### Data preparation

The first step that we are going to do for data preperation is text-clening. We are going to remove punctuations and lowercase all words.

In [6]:
def clean_text(txt):
  txt = "".join(v for v in txt if v not in string.punctuation).lower()
  txt = txt.encode('utf8').decode('ascii', 'ignore')
  return txt
corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression',
 'commuter reprogramming',
 'ford changed leaders looking for a lift its still looking',
 'romney failed to win at utah convention but few believe hes doomed',
 'chain reaction',
 'he forced the vatican to investigate sex abuse now hes meeting with pope francis']

### Generating Sequence of N-grams Tokens
Language modelling requires a sequence input data, as given a sequence of word-tokens the aim is to predict the next word.

### Tokenization

In [7]:
tokenizer = keras.preprocessing.text.Tokenizer()

def get_sequence_of_tokens(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
  input_sequences = []
  for line in corpus:
    token_list= tokenizer.texts_to_sequences([
        line
    ])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)
  return input_sequences, total_words

In [8]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:28]

[[391, 915],
 [391, 915, 2484],
 [391, 915, 2484, 5166],
 [391, 915, 2484, 5166, 664],
 [391, 915, 2484, 5166, 664, 78],
 [391, 915, 2484, 5166, 664, 78, 7],
 [391, 915, 2484, 5166, 664, 78, 7, 2],
 [391, 915, 2484, 5166, 664, 78, 7, 2, 475],
 [391, 915, 2484, 5166, 664, 78, 7, 2, 475, 11],
 [391, 915, 2484, 5166, 664, 78, 7, 2, 475, 11, 5167],
 [331, 3],
 [331, 3, 5168],
 [331, 3, 5168, 2],
 [331, 3, 5168, 2, 12],
 [331, 3, 5168, 2, 12, 1011],
 [331, 3, 5168, 2, 12, 1011, 22],
 [331, 3, 5168, 2, 12, 1011, 22, 736],
 [331, 3, 5168, 2, 12, 1011, 22, 736, 332],
 [331, 3, 5168, 2, 12, 1011, 22, 736, 332, 412],
 [331, 3, 5168, 2, 12, 1011, 22, 736, 332, 412, 5],
 [331, 3, 5168, 2, 12, 1011, 22, 736, 332, 412, 5, 5169],
 [1, 12],
 [1, 12, 5170],
 [1, 12, 5170, 3366],
 [14, 2],
 [14, 2, 3367],
 [14, 2, 3367, 4],
 [14, 2, 3367, 4, 562]]

### How ngrams works?

```
[1, 12] -> this is
[1, 12, 5170] -> this is a
[1, 12, 5170, 3366]-> this is a dog
```

### Padding sequences and generating target.

Sequences must be of the same length. Ww will create n-grams sequences as predictors and the next word of the n-grams as label.

```
they -> are (label)
they are -> standing (label)
they are standing -> at (label)
they are standing at -> the (label)
they are standing at the -> door (label)
```

In [41]:
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, 
                                            padding='pre'))
  predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
  label = keras.utils.to_categorical(label, num_classes=total_words)
  return predictors, label, max_sequence_len

In [42]:
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

### LSTM for text generation

Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. This state allows the neurons an ability to remember what have been learned so far.

The memory state in RNNs gives an advantage over traditional neural networks but a problem called Vanishing Gradient is associated with them. In this problem, while learning with a large number of layers, it becomes really hard for the network to learn and tune the parameters of the earlier layers. To address this problem, A new type of RNNs called LSTMs (Long Short Term Memory) Models have been developed.

LSTMs have an additional state called ‘cell state’ through which the network makes adjustments in the information flow. The advantage of this state is that the model can remember or forget the leanings more selectively. To learn more about LSTMs, here is a great post. Lets architecture a LSTM model in our code.



In [44]:
predictors

array([[   0,    0,    0, ...,    0,    0,  391],
       [   0,    0,    0, ...,    0,  391,  915],
       [   0,    0,    0, ...,  391,  915, 2484],
       ...,
       [   0,    0,    0, ..., 3044,    4,   91],
       [   0,    0,    0, ...,    4,   91,   13],
       [   0,    0,    0, ...,   91,   13,    1]], dtype=int32)

In [45]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = keras.Sequential(name="text-generator")
  
  # Add Input Embedding Layer
  model.add(keras.layers.Embedding(total_words, 100,
                                    input_length=input_len))
  
  # Add Hidden Layer 1 - LSTM Layer
  model.add(keras.layers.LSTM(100))
  model.add(keras.layers.Dropout(0.1))
  
  # Add Output Layer
  model.add(keras.layers.Dense(total_words, activation='softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=["acc"])
  return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "text-generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 23, 100)           1126500   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 11265)             1137765   
Total params: 2,344,665
Trainable params: 2,344,665
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.fit(predictors, label,
          epochs=5, verbose=1,
          shuffle=True,
          validation_split=.1
          )

Epoch 1/5
1457/1457 [==============================] - 25s 16ms/step - loss: 7.8566 - acc: 0.0383 - val_loss: 7.6306 - val_acc: 0.0541
Epoch 2/5
1457/1457 [==============================] - 22s 15ms/step - loss: 7.3522 - acc: 0.0544 - val_loss: 7.5598 - val_acc: 0.0657
Epoch 3/5
1457/1457 [==============================] - 22s 15ms/step - loss: 7.0050 - acc: 0.0701 - val_loss: 7.5182 - val_acc: 0.0840
Epoch 4/5
1457/1457 [==============================] - 22s 15ms/step - loss: 6.6334 - acc: 0.0905 - val_loss: 7.5503 - val_acc: 0.0914
Epoch 5/5
1457/1457 [==============================] - 22s 15ms/step - loss: 6.2566 - acc: 0.1102 - val_loss: 7.6219 - val_acc: 0.0945


### Generating text.
Great, our model architecture is now ready and we can train it using our data. Next lets write the function to predict the next word based on the input words (or seed text). We will first tokenize the seed text, pad the sequences and pass into the trained model to get predicted word. The multiple predicted words can be appended together to get predicted sequence.

In [48]:
def generate_text(seed_text, max_len, model, max_sequence_len):
  for _ in range(max_len):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list],
                               maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted = np.argmax(predicted, axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
  return seed_text.title()

In [49]:
print (generate_text("united states", 5, model, max_sequence_len))
print (generate_text("preident trump", 4, model, max_sequence_len))
print (generate_text("donald trump", 4, model, max_sequence_len))
print (generate_text("india and china", 4, model, max_sequence_len))
print (generate_text("new york", 4, model, max_sequence_len))
print (generate_text("science and technology", 5, model, max_sequence_len))

United States And The New York Times
Preident Trump And The New York
Donald Trump And The New York
India And China And The New York
New York Today The New York
Science And Technology A New York Times Is


### Conclusion
We have leant how to use LSTM for text generation by expanding [this kaggle](https://www.kaggle.com/garidziracrispen/beginners-guide-to-text-generation-using-lstms) notebook. Next we will try to implement this idea and try to create a model that will be a little bit accurate than this one.